In [1]:
import gym
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# IS_SLIPPERY TRUE : 미끄러운 환경 조성 
env = gym.make('FrozenLake-v0')
#오른쪽으로 지시해도 가지 못함 (왜냐하면 미끄러졌기 때문에)

[2017-04-17 21:56:52,961] Making new env: FrozenLake-v0


In [3]:
tf.reset_default_graph()

In [4]:
inputOne = tf.placeholder(shape = [1,16], dtype=tf.float32)
W = tf.Variable(tf.random_uniform([16,4],0,0.01))
Q_output = tf.matmul(inputOne,W)
predict = tf.argmax(Q_output,1)

In [5]:
# 손실(Loss)를 최적화 시키는 코드 
#placeholder 껍데기 생성 
Q_next = tf.placeholder(shape = [1,4], dtype=tf.float32)
#손실 연산 ==> 손실이 최소로 가는 방향까지 계산하는게 중요
loss = tf.reduce_sum(tf.square(Q_next - Q_output))
#학습 최적화 과정 
train = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModels = train.minimize(loss)
"""아담 최적화를 적용할 경우 주석 해제"""
#ModelAdam = tf.train.AdamOptimizer(0.0001).minimize(loss)

'아담 최적화를 적용할 경우 주석 해제'

In [6]:
#변수 초기화 => 꼭 필요한 과정 
init = tf.global_variables_initializer()

In [ ]:
y = .99
e = 0.1
num_episodes = 2000 # 2000번 학습 
jList = []
rList = []

#텐서플로우는 항상 session이란 메소드에 넣어서 실행해야 동작함.
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        #gym 환경 초기화 ( 중요 )
        s = env.reset()
        rAll = 0
        d = False
        j = 0
        while j < 99:
            j+=1
            
            a,allQ = sess.run([predict,Q_output],feed_dict={inputOne:np.identity(16)[s:s+1]})
            
            if np.random.rand(1) < e:
                a[0] = env.action_space.sample()  # take a random action
                
            #환경으로 부터 새로운 상태와 보상을 얻는다. 
            s1,r,d,_ = env.step(a[0])
            # Q^ 값을 얻어온다. 
            Q1 = sess.run(Q_output,feed_dict={inputOne:np.identity(16)[s1:s1+1]})
            # 선택된 행동으로 부터 최대값의 Q값을 가져옴
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,a[0]] = r + y*maxQ1
            
            #Train our network using target and predicted Q values
            _,W1 = sess.run([updateModels,W],feed_dict={inputOne:np.identity(16)[s:s+1],Q_next:targetQ})
            rAll += r
            s = s1
            
            if d == True:
                #Reduce chance of random action as we train the model.
                e = 1./((i/50) + 10)
                break
                
        jList.append(j)
        rList.append(rAll)
print ("Percent of succesful episodes: " + str(sum(rList)/num_episodes) + "%")

In [ ]:
zs = 99.

In [ ]:
print(zs)